# Demonstrating PDF Journalling - Part 1: Basics
Journalling updates to a PDF is supported with MuPDF v1.19.*.

Journalling is a logging mechanism which permits either **reverting** or **re-applying** changes to a PDF.

Similar to LUWs "Logical Units of Work" in modern database systems, one can group a set of updates into an "operation". In MuPDF journalling, an *operation* plays the role of a LUW.

In [ ]:
import fitz

if tuple(map(int, fitz.VersionBind.split("."))) < (1, 19, 0):
    raise ValueError("Need PyMuPDF v1.19.0 or higher")

doc = fitz.open()  # work with an empty PDF
doc.journal_enable()  # enable journalling for it

After journalling is enabled:

> **_All_** updates are logged and must be executed within the range of some **operation**.

So we first try an update without having an active operation:

In [ ]:
try:
    page = doc.new_page()
except RuntimeError:
    print("This does not work without an active operation!")


In [ ]:
doc.journal_start_op("op1")  # define start of an operation
page = doc.new_page()
doc.journal_stop_op()  # define stop of operation

This time it worked: we have a page. Now insert three lines of text - each within its own operation, so they can be individually undone (rolled back).

In [ ]:
for i in range(3):
    doc.journal_start_op("Line-%i" % i)
    # insert next line 20 points below previous one
    page.insert_text((100, 100 + 20*i), "This is line %i." % i)
    doc.journal_stop_op()

Now let us have a look at some journal information.
1. What is our current operation number and total number of operations?

In [ ]:
doc.journal_position()  # returns the tuple: (current op number, total op count)

2. What journalling activity can we do at this point: undo? redo?

In [ ]:
doc.journal_can_do()

Only undo activities are possible, currently. Let us try one, but first we look at what is on the page:

In [ ]:
print(page.get_text())  # this should show three lines of text

Undo an operation and display journalling status information like above:

In [ ]:
doc.journal_undo()  # undo an operation
doc.journal_position()  # where are we in the journal?


In [ ]:
doc.journal_can_do()  # what can we do now?

Now confirm what happened to the page content. If all worked fine, we should see only 2 lines of text.

In [ ]:
print(page.get_text())

Good! Last text insertion was reverted. We change our mind again and redo (re-apply) the undone operation.

In [ ]:
doc.journal_redo()  # redo reverted operation
doc.journal_position()

In [ ]:
doc.journal_can_do()

Confirm that we have three text lines again:

In [ ]:
print(page.get_text())

The journal data itself is kept in an area maintained by MuPDF code. The journal can be saved to a file or to a Python file-like object with a binary output option. If journalled updates are executed against an **_existing PDF_** (not a new and empty one as we did here), the journal file, together with a specially saved PDF "snapshot" can be used to continue updating that PDF - including undo / redo operations.

Here we display our update session to see the type of information stored.

In [ ]:
import io
fp = io.BytesIO()  # we need a file object with binary output capability
doc.journal_save(fp)  # save journal file
# display the file content (decode the bytes object for a nicer look)
print(fp.getvalue().decode())